In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
'''for i in range (1,18):
    vars()['ecg'+str(i)]=MITBIH(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
'''
smpl_rte=480                               #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
cls_num=17
#ecg=np.array([])
ecg=np.empty((0,smpl_rte+1), float)
#base_wn_dt=np.array([])
windws=np.array([])

for cls in range (1,18):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['ecg0'+str(cls)]=MITBIH(cls)         #فراخوانی داده های اصلی
    dta=np.array(vars()['ecg0'+str(cls)][int(.1*len(vars()['ecg0'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng_sig(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   
    vars()['ecg'+str(cls)]=np.array(vars()['wndws'+str(cls)])
    ecg=np.concatenate((ecg,vars()['ecg'+str(cls)]),axis=0)    
    #base_wn_dt=np.append(base_wn_dt,vars()['wndws'+str(cls)])
wdth=int(len(vars()['wndws'+str(cls)][0]))
#base_wn_dt=np.reshape(base_wn_dt,(int(len(base_wn_dt)/wdth),wdth))  #base data windows
os.chdir(cwd)

In [3]:
np.shape(ecg1)

(2598, 481)

In [4]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

'''furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))'''

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (2598, 481)
shape base_wn= (10140, 481)
smpl_rte= 480
mx_wndws= 2598


In [6]:
i=cls=11
mthd=70
vlum_sig=10000
adrs=cwd+'/AugEvl0/AugEvl'+str(i)+str(mthd)+'.csv'
dta0 = np.array(pd.read_csv(adrs))             # Load mat file data into data.

In [7]:
LSTM_new=np.append(dta0[:,1:-5],GDO_rndm_beat(ecg,i,int(vlum_sig),mag=.1),axis=0)
np.shape(LSTM_new)

(11814, 481)

In [8]:
LSTM_new[0]

array([1196.82095253, 1197.39902323, 1190.04339473, 1192.50719866,
       1189.05388132, 1183.28373598, 1171.29121114, 1164.15060483,
       1145.41765752, 1135.89340999, 1132.31020505, 1115.72452771,
       1104.19662038, 1086.91008497, 1076.39970896, 1064.49889793,
       1054.09463108, 1049.89811844, 1043.71441551, 1035.23129282,
       1024.69176259, 1019.23776127, 1017.27931142, 1017.93164363,
       1017.89017698, 1017.15539879, 1014.72318867, 1012.95015483,
       1009.75238768, 1005.88409902, 1006.54901388, 1002.13592433,
        997.84842658,  991.86330513,  983.6786089 ,  976.2231937 ,
        969.79078838,  967.84367646,  963.70705538,  969.715575  ,
        981.31100963,  998.00996744, 1013.18252995, 1022.90521743,
       1030.3813317 , 1033.21375234, 1034.39971422, 1033.53073028,
       1034.8074789 , 1039.22136167, 1045.81710336, 1055.88844311,
       1057.65183075, 1058.66544187, 1063.84301152, 1063.67517709,
       1066.65376524, 1065.08274712, 1061.69441987, 1059.19833

In [9]:
clm0vlu=np.zeros(len(LSTM_new))
cls_clm=np.array(clm0vlu)
cls_clm=np.int16(cls_clm+cls)
cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
for j in range (5):
    LSTM_new=np.concatenate((LSTM_new,cls_clm),axis=1)

In [10]:
np.shape(LSTM_new)

(11814, 486)

In [11]:
pd.DataFrame(LSTM_new).to_csv('{}/AugEvl1/AugEvl{}{}.csv'.format(cwd,cls,mthd+1))